In [0]:
# Install a Drive FUSE wrapper.
# https://github.com/astrada/google-drive-ocamlfuse
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse

In [0]:
# Generate auth tokens for Colab
from google.colab import auth
auth.authenticate_user()

In [0]:
# Generate creds for the Drive FUSE library.
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

In [0]:
# Create a directory and mount Google Drive using that directory.
!mkdir -p drive
!google-drive-ocamlfuse drive

In [0]:
# Clone repositories
!git clone https://github.com/dhgrs/download_dataset.git
!git clone https://github.com/dhgrs/chainer-VQ-VAE.git

In [0]:
# Download dataset
!python download_dataset/download_CMU_ARCTIC.py .

In [0]:
# Install packages
!apt -y install libcusparse8.0 libnvrtc8.0 libnvtoolsext1
!ln -snf /usr/lib/x86_64-linux-gnu/libnvrtc-builtins.so.8.0 /usr/lib/x86_64-linux-gnu/libnvrtc-builtins.so
!pip install 'cupy-cuda80==4.0.0b4' 'chainer==4.0.0b4'
!pip install librosa

In [0]:
# Set dataset directory
!sed -i -e "s/root = 'VCTK-Corpus'/root = 'CMU_ARCTIC'/g" chainer-VQ-VAE/opt.py
!sed -i -e "s/dataset = 'VCTK'/dataset = 'ARCTIC'/g" chainer-VQ-VAE/opt.py
!sed -i -e "s/snapshot_interval = (1000, 'iteration')/snapshot_interval = (5000, 'iteration')/g" chainer-VQ-VAE/opt.py
!cat chainer-VQ-VAE/opt.py

In [0]:
# Set result directory
!sed -i -e "s/datetime.datetime/'drive\/' + datetime.datetime/g" chainer-VQ-VAE/train.py
!cat chainer-VQ-VAE/train.py

In [0]:
# Copy files into current directory
!cp chainer-VQ-VAE/*.py .

In [0]:
!python train.py -g 0
# If you want to resume the training from snapshot on google drive, like below.
# !python train.py -g 0 -r drive/2018_03_01_01_23_45/snapshot_iter_5000